In [140]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = 999
import numpy as np

In [1]:
# import os
# os.getcwd()

'/Users/yiweiluo/scientific-debates/cc_stance'

In [2]:
def to_int(x):
    if x == 'neutral' or x == 2:
        return -1
    elif x == 'disagree' or x == 1:
        return 0
    else:
        return 1

In [3]:
BERT_PRED_DIR = 'BERT/bert_preds/'
DATA_DIR = './data_creation/scripts/save/'

# BERT+classifier predictions

In [17]:
def do_error_analysis(pred_filename,true_filename):
    # Load and pre-process
    preds = pd.read_csv(BERT_PRED_DIR+pred_filename,sep='\t',header=0,index_col=False)
    true = pd.read_csv(DATA_DIR+true_filename,sep='\t',header=None,index_col=False)
    assert len(preds) == len(true)
    preds['true'] = preds['true'].apply(to_int)
    preds['predicted'] = preds['predicted'].apply(to_int)
    true[1] = true[1].apply(to_int)
    print(true.head(10))
    assert np.sum([(true[1] == preds['true'])]) == len(preds)
    preds['sentence'] = true[0]
    
    bad_preds = preds[preds.true != preds.predicted]
    good_preds = preds[preds.true == preds.predicted]
    opp_preds = bad_preds[bad_preds.true == bad_preds.predicted*-1]
    N_correct = len(good_preds)
    N_wrong = len(bad_preds)
    N_opp = len(opp_preds)
    total_N = len(preds)
    print('Num correct: {},\tNum wrong: {},\tNum opposite: {}\n'.format(N_correct,
                                                                N_wrong,
                                                                N_opp))
    print('Percent correct: {},\tPercent wrong: {},\tPercent opposite: {}\n'\
          .format(round(N_correct/len(preds),3),
                 round(N_wrong/len(preds),3),
                 round(N_opp/len(preds),3)))
    
    prop_opp_from_agrees = round(len(opp_preds[opp_preds.true == 1])/len(opp_preds)*100,2)
    print('Of the opposite preds, {}% were true agrees and {}% were true disagrees.\n'\
         .format(prop_opp_from_agrees,100-prop_opp_from_agrees))
    
    false_neutrals = bad_preds[bad_preds.predicted == 0]
    false_agrees = bad_preds[bad_preds.predicted == 1]
    false_disagrees = bad_preds[bad_preds.predicted == -1]
    true_neutrals = good_preds[good_preds.predicted == 0]
    true_agrees = good_preds[good_preds.predicted == 1]
    true_disagrees = good_preds[good_preds.predicted == -1]
    
    print('Proportion of false agrees: {} (out of {} true agrees)'.\
          format(round(len(false_agrees)/len(preds),3),
                len(true_agrees)))
    print('Distribution of true values of false agrees:\n', 
          false_agrees.true.value_counts())
    
    print('Proportion of false disagrees: {} (out of {} true disagrees)'.\
          format(round(len(false_disagrees)/len(preds),3),
                len(true_disagrees)))
    print('Distribution of true values of false disagrees:\n', 
          false_disagrees.true.value_counts())
    
    print('Proportion of false neutrals: {} (out of {} true neutrals)'.\
          format(round(len(false_neutrals)/len(preds),3),
                len(true_neutrals)))
    print('Distribution of true values of false neutrals:\n', 
          false_neutrals.true.value_counts())
    
    return preds,opp_preds,bad_preds

## Windowed training data (MTurk), window size = 1

In [23]:
window_1_preds_dev,window_1_opp_preds_dev,window_1_bad_preds_dev = do_error_analysis('windowed_mturk_1_dev_preds.tsv',
                 'mturk_windowed_1/dev.tsv')

                                                                                                                                                                                                      0  \
0  If carbon dioxide emissions continue to rise beyond 2020, or even remain level, the temperature goals set in Paris become almost unattainable.                                                         
1  45 % of the general public view perceived global warming as caused by humans.                                                                                                                          
2  Carbon dioxide will hit another record high in 201.                                                                                                                                                    
3  Four in ten Americans (40%) have personally experienced the effects of global warming.                                                                                                   

In [24]:
window_1_preds_test,window_1_opp_preds_test,window_1_bad_preds_test = do_error_analysis('windowed_mturk_1_test_preds.tsv',
                 'mturk_windowed_1/test.tsv')

                                                                                                                                                                            0  \
0  Warmer-than-normal sea surface temperatures are a key player in the development of hurricanes such as Katrina and superstorm Sandy, which hit the U.S. east coast in 2011.   
1  We will continue to rely in part on fossil fuels while we transition to a low-carbon economy .                                                                               
2  Some parts of the world the world, such as the coast of California, saw sea levels fall.                                                                                     
3  Tackling climate change will be the greatest global health opportunity of the 21st century .                                                                                 
4  The West Antarctic ice sheet could disappear in as little as 1,000 years.                                       

In [26]:
window_1_opp_preds_test

,true,predicted,sentence
9,1,-1,They are too pessimistic about the chances of limiting global warming.
10,-1,1,"Global warming may be occurring more slowly than earlier thought, and that previous climate models may have been on the hot side."
20,1,-1,Co2 is a demon molecule that causes global warming.
26,-1,1,Global warming is overestimated by 100 % over 20 years.
27,-1,1,Global warming global warming is natural.
29,-1,1,The global warming of the 1900s is caused by a rise in solar output.
41,-1,1,From 1860-1880 as well as from 1910-1940 the rate of global warming is the same as from 1980-200.
44,-1,1,Human activities are not a major cause of global warming.
53,1,-1,This is evidence of man-made global warming.
58,-1,1,Global cooling is coming.


In [25]:
window_1_opp_preds_test['sentence'].values

array(['They are too pessimistic about the chances of limiting global warming.',
       'Global warming may be occurring more slowly than earlier thought, and that previous climate models may have been on the hot side.',
       'Co2 is a demon molecule that causes global warming.',
       'Global warming is overestimated by 100 % over 20 years.',
       'Global warming global warming is natural.',
       'The global warming of the 1900s is caused by a rise in solar output.',
       'From 1860-1880 as well as from 1910-1940 the rate of global warming is the same as from 1980-200.',
       'Human activities are not a major cause of global warming.',
       'This is evidence of man-made global warming.',
       'Global cooling is coming.',
       'The fifteen-year long “ global warming ” campaign all along meant “ climate change ” and that this in turn means that places supposed to get hotter get hotter and that places that are supposed to get colder — under global warming, er, climate ch

In [129]:
window_1_preds_downsampled_mturk_test,\
window_1_opp_preds_downsampled_mturk_test,\
window_1_bad_preds_downsampled_mturk_test = do_error_analysis('windowed_1_mturk_model_eval_on\
_downsampled_test_preds.tsv',
                 'downsampled_mturk/test.tsv')

                                                                                                                                                                                                                                                                                                                               0  \
0  The ridiculously warm temperatures in the Arctic during October and November this year this year are off the charts over our 68 years of measurements.                                                                                                                                                                          
1  No one can say they are happy about climate change.                                                                                                                                                                                                                                                                             
2  Earlier this year painted

In [146]:
window_1_preds_downsampled_mturk_dev,\
window_1_opp_preds_downsampled_mturk_dev,\
window_1_bad_preds_downsampled_mturk_dev = do_error_analysis('windowed_1_mturk_model_eval_on\
_downsampled_dev_preds.tsv',
                 'downsampled_mturk/dev.tsv')

                                                                                                                                                                                                                                                 0  \
0  The effects of decisions like these will be felt for generations as global warming leads to more powerful storms and higher risk of damage to property.                                                                                           
1  Hawaii is committed to environmental stewardship, and we look forward to working with other states to fight global climate change.                                                                                                                
2  The global south in particular is both more vulnerable to the effects of climate change and less equipped to deal with those effects.                                                                                                             
3  They would lo

## Windowed training data (MTurk), window size = 2

In [41]:
window_2_preds_dev,window_2_opp_preds_dev,window_2_bad_preds_dev = do_error_analysis('windowed_mturk_dev_preds.tsv',
                 'mturk_windowed_2/dev.tsv')

                                                                                                                                                                                                      0  \
0  If carbon dioxide emissions continue to rise beyond 2020, or even remain level, the temperature goals set in Paris become almost unattainable.                                                         
1  45 % of the general public view perceived global warming as caused by humans.                                                                                                                          
2  Carbon dioxide will hit another record high in 201.                                                                                                                                                    
3  Four in ten Americans (40%) have personally experienced the effects of global warming.                                                                                                   

In [30]:
window_2_opp_preds_dev

,true,predicted,sentence
6,-1,1,By now there should have been global warming equivalent to 2.8 degrees / century.
16,1,-1,Global warming is what is causing the climate to change .
20,1,-1,The United States government has violated their constitutional rights by allowing fossil fuel production to continue — despite knowing its effects on the planet.
23,1,-1,The basics are clear long ago and the response to global warming is more about ethics and economics than data.
38,1,-1,Global warming global warming global warming global warming turns out to be as real and present and dangerous as ever it is.
45,-1,1,The water level is affected by things other than global warming.
48,-1,1,Global warming causes increased human lifespans.
60,-1,1,Flat hurricanes have not become more numerous and stronger due to carbon pollution.
61,1,-1,"The beans contains some powerful refutations of a number of other global warming denier talking points, as well as some ingenious sequences that explain the planetary-scale significance of climate change."
71,1,-1,— Climate change is causing significant social unrest — is one that is being repeated with greater frequency by leading proponents of the theory humans are primarily responsible for causing global warming.


In [40]:
window_2_preds_test,window_2_opp_preds_test,window_2_bad_preds_test = do_error_analysis('windowed_mturk_test_preds.tsv',
                 'mturk_windowed_2/test.tsv')

                                                                                                                                                                            0  \
0  Warmer-than-normal sea surface temperatures are a key player in the development of hurricanes such as Katrina and superstorm Sandy, which hit the U.S. east coast in 2011.   
1  We will continue to rely in part on fossil fuels while we transition to a low-carbon economy .                                                                               
2  Some parts of the world the world, such as the coast of California, saw sea levels fall.                                                                                     
3  Tackling climate change will be the greatest global health opportunity of the 21st century .                                                                                 
4  The West Antarctic ice sheet could disappear in as little as 1,000 years.                                       

In [32]:
window_2_opp_preds_test

,true,predicted,sentence
9,1,-1,They are too pessimistic about the chances of limiting global warming.
19,1,-1,"Anyone objectively looking at the data, even at the graph produced above, can see that climate models are much, much, much better at predicting global warming over the past decade."
20,1,-1,Co2 is a demon molecule that causes global warming.
26,-1,1,Global warming is overestimated by 100 % over 20 years.
41,-1,1,From 1860-1880 as well as from 1910-1940 the rate of global warming is the same as from 1980-200.
...,...,...,...
428,1,-1,Climate change is an absolute first-order issue.
434,-1,1,The climate is always changing in accordance with natural causes and recent changes.
441,1,-1,What will you tell your children is the reason to fail and leave them facing a climate chaos that you knowingly brought upon them.
447,-1,1,"Young people like Thunberg are simply being used as ""human shields"" for adult climate activists who recognize that the climate scare will soon lose credibility."


## Flattened training data (MTurk)

In [39]:
flattened_preds_dev,flattened_opp_preds_dev,flattened_bad_preds_dev = do_error_analysis('flattened_mturk_dev_preds.tsv',
                 'mturk_flat/dev.tsv')

                                                                                                                                                                                                0  \
0  Global warming is mainly caused by human activity.                                                                                                                                               
1  Humans are causing global warming.                                                                                                                                                               
2  Since 1978, the winter Arctic ice cap has shrunk by 12 percent per decade.                                                                                                                       
3  The paper in no way diminishes the extensive body of observations that global warming is happening and that it is largely due to human activity.                                                 
4  One round-tr

In [6]:
opp_preds_dev

,true,predicted,sentence
3,1,-1,The paper in no way diminishes the extensive body of observations that global warming is happening and that it is largely due to human activity.
46,1,-1,The powerful of the world ignore the immense devastation wrought by pollution in favor of an ideologically driven crusade against global warming.
53,1,-1,"Global warming causes more prostitution, barroom brawls, airline turbulence."
73,1,-1,"Hurricanes are among the extreme weather events that have been exacerbated by manâs selfishness, greed, and refusal to amend his carbon-guzzling lifestyle."
88,1,-1,You have failed in humanity's task of protecting the climate.
208,-1,1,Global warming global warming is natural.
211,-1,1,Global warming is a relatively distant threat.
217,-1,1,"While the world is in turmoil and falling apart in so many different ways especially with ISIS, our President is worried about global warming–what a ridiculous situation."
228,-1,1,The hurricane isnât âentirely a product of climate changeâ and severe weather has existed for eons before fossil fuels were used.
229,-1,1,The findings will infuriate environmental campaigners who argue by 2030 we could be facing increased sea levels and flooding due to glacial melt at the poles.


In [38]:
flattened_preds_test,flattened_opp_preds_test,flattened_bad_preds_test = do_error_analysis('flattened_mturk_test_preds.tsv',
                 'mturk_flat/test.tsv')

                                                                                                                                                                                                                                        0  \
0  The level of carbon dioxide in the atmosphere has never been higher.                                                                                                                                                                     
1  Human emissions of greenhouse gases are causing global warming.                                                                                                                                                                          
2  Global warming will harm Americans.                                                                                                                                                                                                      
3  Progress in avoiding environmental regrets can co

In [9]:
opp_preds_test

,true,predicted,sentence
4,1,-1,And that’s not the only way global warming causes psychological problems.
25,1,-1,"Global warming, unsurprisingly, is bad news for snowpack."
54,1,-1,Global warming is to blame.
58,1,-1,The issue of global warming's the height of arrogance for human beings to think that they can control the weather.
84,1,-1,"Ice at both ends of the Earth is melting far faster, and oceans are warming far faster, than previously thought, suggesting that scientific estimates of a warming planet have been too conservative."
117,1,-1,Beating climate change won't require a miracle.
315,-1,1,"It gives us a dangerous climate that we make safe, above all, by using energy from fossil fuels."
318,-1,1,Global warming has been at a plateau since 1990.
329,-1,1,"All that our world leaders need is a little more common sense, in order to avoid the nonsense of seeking a globally-binding treaty to reduce CO2 emissions."
330,-1,1,The world will avoid global warming of more than 2C above pre-industrial levels.


## Downsampled vanilla MTurk training

In [37]:
downsampled_preds_dev,downsampled_opp_preds_dev,downsampled_bad_preds_dev = do_error_analysis('downsampled_mturk_dev_preds.tsv',
                 'downsampled_mturk/dev.tsv')

                                                                                                                                                                                                                                                 0  \
0  The effects of decisions like these will be felt for generations as global warming leads to more powerful storms and higher risk of damage to property.                                                                                           
1  Hawaii is committed to environmental stewardship, and we look forward to working with other states to fight global climate change.                                                                                                                
2  The global south in particular is both more vulnerable to the effects of climate change and less equipped to deal with those effects.                                                                                                             
3  They would lo

In [79]:
# Train on downsampled MTurk; evaluate on dev MTurk
opp_preds

,true,predicted,sentence
31,1,-1,They are too pessimistic about the chances of limiting global warming.
45,1,-1,Its appropriate to call such storms global warming type events.
167,-1,1,The magnitude of the dryness is well beyond what global warming would be able to do so far.
171,-1,1,Global warming is occurring more slowly than computer models forecast a decade ago.
185,-1,1,Global warming can be tackled without changing entrenched patterns of production and consumption in industrialized societies.
188,-1,1,"Ultimately, global warming has become the sole focus of so much environmental discussion that it risks eclipsing much more pressing and demonstrable environmental problems."
189,-1,1,Mitch McConnell and the Republican Party are going to boldly confront climate change and create millions of jobs in the process.
207,-1,1,By now there should have been global warming equivalent to 2.8 degrees / century.
210,-1,1,Policies designed to curb global warming would hurt the economy.
225,-1,1,"All that our world leaders need is a little more common sense, in order to avoid the nonsense of seeking a globally-binding treaty to reduce CO2 emissions."


In [36]:
downsampled_preds_test,downsampled_opp_preds_test,downsampled_bad_preds_test = do_error_analysis('downsampled_mturk_test_preds.tsv',
                 'downsampled_mturk/test.tsv')

                                                                                                                                                                                                                                                                                                                               0  \
0  The ridiculously warm temperatures in the Arctic during October and November this year this year are off the charts over our 68 years of measurements.                                                                                                                                                                          
1  No one can say they are happy about climate change.                                                                                                                                                                                                                                                                             
2  Earlier this year painted

In [82]:
opp_preds_test

,true,predicted,sentence
0,1,-1,The ridiculously warm temperatures in the Arctic during October and November this year this year are off the charts over our 68 years of measurements.
5,1,-1,Living like climate change is real and that we can do something about it are signals to others.
10,1,-1,The low number is derived from a method that did not take account of the way global warming had sped up since the 1970s.
37,1,-1,"It does not include a discussion on climate change, which is an outrage."
87,1,-1,No use in denying global warming exists.
89,1,-1,"Global warming is the tide coming in, and El Nio is the waves."
102,1,-1,"The beans contains some powerful refutations of a number of other global warming denier talking points, as well as some ingenious sequences that explain the planetary-scale significance of climate change."
115,1,-1,The missed deadline underscores the disdain with which the Trump administration views the worldwide effort to rein in global warming.
116,1,-1,"Today 's [ Green New Deal ] vote is a partisan stunt to side-step needed debate on climate action, and give Republicans cover to put oil lobby checks over our kids."
146,1,-1,"Global warming is real, because the rise of terrorism tracks with their adjusted climate data."


## Vanilla MTurk only trained model

BERT trained on MTurk only, no downsampling:

In [42]:
vanilla_mturk_preds_dev,vanilla_mturk_opp_preds_dev,vanilla_mturk_bad_preds_dev = do_error_analysis('mturk_only_dev_preds.tsv',
                 'mturk_only/dev.tsv')

FileNotFoundError: [Errno 2] File b'./data_creation/scripts/save/mturk_only/dev.tsv' does not exist: b'./data_creation/scripts/save/mturk_only/dev.tsv'

More of the opposite predictions are coming from true "disagrees" than "agrees".

In [43]:
len(opp_preds[opp_preds.true == 1])/len(opp_preds)

0.18181818181818182

In [44]:
len(opp_preds[opp_preds.true == -1])/len(opp_preds)

0.8181818181818182

In [58]:
opp_preds

,true,predicted,sentence
31,1,-1,They are too pessimistic about the chances of limiting global warming.
45,1,-1,Its appropriate to call such storms global warming type events.
167,-1,1,The magnitude of the dryness is well beyond what global warming would be able to do so far.
171,-1,1,Global warming is occurring more slowly than computer models forecast a decade ago.
185,-1,1,Global warming can be tackled without changing entrenched patterns of production and consumption in industrialized societies.
188,-1,1,"Ultimately, global warming has become the sole focus of so much environmental discussion that it risks eclipsing much more pressing and demonstrable environmental problems."
189,-1,1,Mitch McConnell and the Republican Party are going to boldly confront climate change and create millions of jobs in the process.
207,-1,1,By now there should have been global warming equivalent to 2.8 degrees / century.
210,-1,1,Policies designed to curb global warming would hurt the economy.
225,-1,1,"All that our world leaders need is a little more common sense, in order to avoid the nonsense of seeking a globally-binding treaty to reduce CO2 emissions."


In [173]:
false_neutrals = bad_preds[bad_preds.predicted == 0]
false_agrees = bad_preds[bad_preds.predicted == 1]
false_disagrees = bad_preds[bad_preds.predicted == -1]
true_neutrals = good_preds[good_preds.predicted == 0]
true_agrees = good_preds[good_preds.predicted == 1]
true_disagrees = good_preds[good_preds.predicted == -1]

In [175]:
len(false_agrees)/len(mturk_only_preds)

0.15030674846625766

In [176]:
len(true_agrees)

84

In [177]:
false_agrees.true.value_counts()

 0    28
-1    21
Name: true, dtype: int64

In [179]:
len(false_disagrees)/len(mturk_only_preds)

0.08895705521472393

In [180]:
len(true_disagrees)

27

In [181]:
false_disagrees.true.value_counts()

0    15
1    14
Name: true, dtype: int64

In [182]:
len(false_neutrals)/len(mturk_only_preds)

0.13496932515337423

In [183]:
len(true_neutrals)

93

In [184]:
false_neutrals.true.value_counts()

 1    35
-1    9 
Name: true, dtype: int64

## All data trained model, eval on MTurk only

BERT Trained on all data:

In [131]:
len(opp_preds[opp_preds.true == 1])/len(opp_preds)

0.25

In [132]:
len(opp_preds[opp_preds.true == -1])/len(opp_preds)

0.75

In [119]:
opp_preds

,true,predicted,sentence
13,-1,1,This a sneaky way to cover up the reality that global warming is a sham.
17,-1,1,An increase in C02 emissions from fossil fuels will be a great benefit to plant and animal life on Earth.
42,-1,1,Global warming idiots out of danger.
49,-1,1,"The world may have already begun to cool in response to a weakening Sun, a phenomenon far more dangerous than any possible human-induced warming."
50,-1,1,Global warming helps plants — and thus also helps humans and animals .
58,-1,1,There is a rush to judgment that every time something happens it’s because of global warming.
60,-1,1,"For example, stressing the idea that human activity is behind global warming can be counterproductive."
86,-1,1,Global warming is a relatively distant threat.
88,-1,1,Satellite data disputes global warming.
98,-1,1,Even in liberal states doubt global warming is caused mostly by human carbon emissions.


In [120]:
false_neutrals = bad_preds[bad_preds.predicted == 0]
false_agrees = bad_preds[bad_preds.predicted == 1]
false_disagrees = bad_preds[bad_preds.predicted == -1]
true_neutrals = good_preds[good_preds.predicted == 0]
true_agrees = good_preds[good_preds.predicted == 1]
true_disagrees = good_preds[good_preds.predicted == -1]

In [121]:
len(false_agrees)/407.

0.3857493857493858

In [122]:
len(true_agrees)

132

In [123]:
false_agrees.true.value_counts()

 0    127
-1    30 
Name: true, dtype: int64

In [124]:
len(false_disagrees)/407

0.07616707616707617

In [125]:
len(true_disagrees)

30

In [126]:
false_disagrees.true.value_counts()

0    21
1    10
Name: true, dtype: int64

In [127]:
len(false_neutrals)/407

0.04668304668304668

In [129]:
len(true_neutrals)

38

In [128]:
false_neutrals.true.value_counts()

 1    13
-1    6 
Name: true, dtype: int64

# Compare wrong predictions across models

In [67]:
model_list = ['downsampled_mturk','mturk_flat','mturk_windowed_1','mturk_windowed_2']

In [44]:
flattened_opp_preds_dev.shape

(15, 3)

In [70]:
all_train_sets = {m: pd.read_csv(DATA_DIR+'{}/train.tsv'.format(m),sep='\t',header=None) 
               for m in model_list}
all_dev_sets = {m: pd.read_csv(DATA_DIR+'{}/dev.tsv'.format(m),sep='\t',header=None) 
               for m in model_list}
all_test_sets = {m: pd.read_csv(DATA_DIR+'{}/test.tsv'.format(m),sep='\t',header=None) 
               for m in model_list}

In [72]:
for ix_m in range(len(model_list)-1):
    for ix_m_2 in range(ix_m+1,len(model_list)):
        print('Test size of {} model:'.format(model_list[ix_m]),
             len(all_test_sets[model_list[ix_m]]))
        print('Test size of {} model:'.format(model_list[ix_m_2]),
             len(all_test_sets[model_list[ix_m_2]]))
        print('Test intersection b/w {} and {}:'.format(model_list[ix_m],
                                                        model_list[ix_m_2]),
             len([s for s in all_test_sets[model_list[ix_m]][1].values if s in 
                 all_test_sets[model_list[ix_m_2]][1].values]))
        print('\n')

Test size of downsampled_mturk model: 861
Test size of mturk_flat model: 369
Test intersection b/w downsampled_mturk and mturk_flat: 861


Test size of downsampled_mturk model: 861
Test size of mturk_windowed_1 model: 464
Test intersection b/w downsampled_mturk and mturk_windowed_1: 861


Test size of downsampled_mturk model: 861
Test size of mturk_windowed_2 model: 464
Test intersection b/w downsampled_mturk and mturk_windowed_2: 861


Test size of mturk_flat model: 369
Test size of mturk_windowed_1 model: 464
Test intersection b/w mturk_flat and mturk_windowed_1: 369


Test size of mturk_flat model: 369
Test size of mturk_windowed_2 model: 464
Test intersection b/w mturk_flat and mturk_windowed_2: 369


Test size of mturk_windowed_1 model: 464
Test size of mturk_windowed_2 model: 464
Test intersection b/w mturk_windowed_1 and mturk_windowed_2: 464




In [71]:
for ix_m in range(len(model_list)-1):
    for ix_m_2 in range(ix_m+1,len(model_list)):
        print('Dev size of {} model:'.format(model_list[ix_m]),
             len(all_dev_sets[model_list[ix_m]]))
        print('Dev size of {} model:'.format(model_list[ix_m_2]),
             len(all_dev_sets[model_list[ix_m_2]]))
        print('Dev intersection b/w {} and {}:'.format(model_list[ix_m],
                                                        model_list[ix_m_2]),
             len([s for s in all_dev_sets[model_list[ix_m]][1].values if s in 
                 all_dev_sets[model_list[ix_m_2]][1].values]),
             len(set(all_dev_sets[model_list[ix_m][1].values])))
        print('\n')

Dev size of downsampled_mturk model: 237
Dev size of mturk_flat model: 246
Dev intersection b/w downsampled_mturk and mturk_flat: 237


Dev size of downsampled_mturk model: 237
Dev size of mturk_windowed_1 model: 309
Dev intersection b/w downsampled_mturk and mturk_windowed_1: 237


Dev size of downsampled_mturk model: 237
Dev size of mturk_windowed_2 model: 309
Dev intersection b/w downsampled_mturk and mturk_windowed_2: 237


Dev size of mturk_flat model: 246
Dev size of mturk_windowed_1 model: 309
Dev intersection b/w mturk_flat and mturk_windowed_1: 246


Dev size of mturk_flat model: 246
Dev size of mturk_windowed_2 model: 309
Dev intersection b/w mturk_flat and mturk_windowed_2: 246


Dev size of mturk_windowed_1 model: 309
Dev size of mturk_windowed_2 model: 309
Dev intersection b/w mturk_windowed_1 and mturk_windowed_2: 309




In [66]:
for ix_m in range(len(model_list)-1):
    for ix_m_2 in range(ix_m+1,len(model_list)):
        print('Train size of {} model:'.format(model_list[ix_m]),
             len(all_train_sets[model_list[ix_m]]))
        print('Train size of {} model:'.format(model_list[ix_m_2]),
             len(all_train_sets[model_list[ix_m_2]]))
        print('Train intersection b/w {} and {}:'.format(model_list[ix_m],
                                                        model_list[ix_m_2]),
             len([s for s in all_train_sets[model_list[ix_m]][1].values if s in 
                 all_train_sets[model_list[ix_m_2]][1].values]))
        print('\n')
# Everything in downsampled model is in flattened model
# Everything in downsampled model is included in training data for window_1, window_2

# Lots of training data overlap

Train size of downsampled model: 942
Train size of flattened model: 6435
Train intersection b/w downsampled and flattened: 942


Train size of downsampled model: 942
Train size of window_1 model: 1267
Train intersection b/w downsampled and window_1: 942


Train size of downsampled model: 942
Train size of window_2 model: 1267
Train intersection b/w downsampled and window_2: 942


Train size of flattened model: 6435
Train size of window_1 model: 1267
Train intersection b/w flattened and window_1: 6435


Train size of flattened model: 6435
Train size of window_2 model: 1267
Train intersection b/w flattened and window_2: 6435


Train size of window_1 model: 1267
Train size of window_2 model: 1267
Train intersection b/w window_1 and window_2: 1267




In [76]:
def get_both_wrong(m1,m2):
    return [s for s in m1['sentence'].values if s in m2['sentence'].values]

In [101]:
def get_m1_right(m1_preds,m1_bad_preds,m2_bad_preds): 
    return [s for s in m1_preds['sentence'].values if 
            s not in m1_bad_preds['sentence'].values and 
           s in m2_bad_preds['sentence'].values]

### Window 1 vs. Window 2

In [152]:
both_wrong = get_both_wrong(window_1_bad_preds_dev,window_2_bad_preds_dev)

In [102]:
window1_right = get_m1_right(window_1_preds_dev,window_1_bad_preds_dev,
                            window_2_bad_preds_dev)

In [104]:
window2_right = get_m1_right(window_2_preds_dev,window_2_bad_preds_dev,
                            window_1_bad_preds_dev)

In [109]:
print(len(window1_right),len(window2_right),len(both_wrong),len(window_1_preds_dev),
len(window_2_preds_dev))

39 29 109 309 309


In [115]:
pd.DataFrame(window2_right)

,0
0,Obama Obama plans to suspend new permits for offshore drilling in Arctic waters at least through the rest of this year.
1,"Two extremely broad propositions – carbon dioxide is a “greenhouse gas,” and human activity has warmed the planet to some extent – tallied up the papers in their survey sample that agreed with these propositions to even the slightest degree, and manufactured the “97 percent consensus” factoid."
2,You can’t stop using fossil fuels without killing people and destroying our economies.
3,Antarctica glacier could collapse within decades and “ sink cities ” after the discovery of a 300-meter doomsday cavity lurking below the ice block.
4,The Kemper County Energy Facility is possible to go on burning the dirtiest of fossil fuels and still make the cuts in carbon dioxide emissions needed to avoid a climate catastrophe.
5,The West Antarctic Ice Sheet is broken — already destabilized by irrevocable melting that foreshadows a slow-motion collapse.
6,"63% of all human-generated carbon emissions have been produced in the past 25 years; that is, nearly two-thirds have been emitted since the first warnings were sounded about what was then called “ global warming ”."
7,By the end of the century the chance of Sandy-level flooding in Lower Manhattan in any given year increases to 50 percent.
8,It is inherently self-serving for the Intergovernmental Panel on Climate Change to identify human activities as the cause of global warming.
9,"For everyone else, there are many, many more pressing concerns, and increased “ extreme weather events ” caused by man-made global warming are an alarmist myth."


In [114]:
pd.DataFrame(window1_right)

,0
0,"If carbon dioxide emissions continue to rise beyond 2020, or even remain level, the temperature goals set in Paris become almost unattainable."
1,The world is in the midst of only the third global coral bleaching event ever recorded.
2,Arctic plant species are extremely responsive to fairly rapid climate shifts in the past.
3,Global warming is what is causing the climate to change .
4,2018 is the fourth hottest year on record
5,"“Climate change” is not a high priority for voters, and they haven’t paid much attention to the latest climate confab in Paris, which had a carbon footprint bigger than a thousand normal Americans living average lives for an entire year."
6,"As the ice sheets melt due to global warming, not only do the ice sheets raise the sea level directly."
7,A mini ice age could hit UK by 2030.
8,The evidence shows “ a fundamental shift in the Earth’s plants ” and that global plant growth should be a central goal for the human race.
9,They are on the verge of passing climate legislation at home and a global carbon accord at the Copenhagen talks.


In [153]:
pd.DataFrame(both_wrong)

,0
0,45 % of the general public view perceived global warming as caused by humans.
1,Four in ten Americans (40%) have personally experienced the effects of global warming.
2,By now there should have been global warming equivalent to 2.8 degrees / century.
3,The United States government has violated their constitutional rights by allowing fossil fuel production to continue — despite knowing its effects on the planet.
4,The rate of Antarctic melt is already six times faster than it is just a few decades ago.
5,The basics are clear long ago and the response to global warming is more about ethics and economics than data.
6,"To further pound the point home, the article the article is festooned with imaginative graphics of future human fossils."
7,The Arctic ice cap is melting faster than is predicted even five years ago.
8,"Despite the increased human emissions of greenhouse gases, between 2002 and 2014, plants are somehow able to absorb more carbon dioxide out of the atmosphere than in previous decades."
9,"The reason why more scientists do n’t advocate putting more carbon dioxide in the atmosphere is because the driving force behind the global warming debate, I hate to say it, isn’t science."


### Window 1 vs. no-window, downsampled

In [132]:
both_wrong = get_both_wrong(window_1_bad_preds_downsampled_mturk_test,
                            downsampled_bad_preds_test)
window1_right = get_m1_right(window_1_preds_downsampled_mturk_test,
                             window_1_bad_preds_downsampled_mturk_test,
                            downsampled_bad_preds_test)
downsampled_right = get_m1_right(downsampled_preds_test,downsampled_bad_preds_test,
                            window_1_bad_preds_downsampled_mturk_test)

In [134]:
print(len(window1_right),len(downsampled_right),len(both_wrong),
      len(window_1_preds_downsampled_mturk_test),
len(downsampled_preds_test))

66 180 112 861 861


In [137]:
66/861. # 7.6% of test examples, window 1 got right and reg. downsampled got wrong.
180/861. # Reg. downsampled got 21% right that window 1 got wrong.
112/861. # For 13% of test examples, both models were wrong.

0.13008130081300814

In [141]:
pd.DataFrame(downsampled_right)

,0
0,"If we are to fully exploit this new oil source, and continue to burn we conventional oil, gas and coal supplies, concentrations of carbon dioxide in the atmosphere eventually would reach levels higher than in the Pliocene era, more than 2.5 million years ago, when sea level is at least 50 feet higher than it is now."
1,You can save the climate with electric cars.
2,"The moment we put real limits and real charges on carbon dioxide emissions, we will see a surge of innovation that will discover even better ways for reducing those emissions."
3,The global average temperature is on pace to increase by 4 degrees Celsius (7.2 degrees Fahrenheit) by the end of the century without some form of concrete action on climate issues.
4,"As the Antarctic ice sheet continues to melt away, we expect multi-meter sea level rise from Antarctica in the coming centuries."
5,Global warming spells bad news for polar bears and other creatures that depend on Arctic sea ice.
6,There is a strong connection between melting sea ice and polar bears dying off.
7,Co2 is responsible for the apparent global temperature increase.
8,"If you've been hearing that extreme cold spells, like the one we're having in the United States now, disproves global warming, don't believe it."
9,"Burning all the worlds deposits of coal, oil and natural gas would raise the temperature enough to melt the entire ice sheet covering Antarctica, driving the level of the sea up by more than 160 feet."


In [142]:
pd.DataFrame(window1_right)

,0
0,No one can say they are happy about climate change.
1,Climate change can be blunted only by moving to renewable energy and capping any growth of fossil fuels.
2,Living like climate change is real and that we can do something about it are signals to others.
3,Human emissions of greenhouse gasses essentially control the future of surface melting in Antarctica.
4,"I am a firm believer that climate change is real, that it is impacted by human behavior and carbon emissions."
5,World sea levels are rising on average.
6,"Climate change could actually make frigid waves of Arctic air more common, a result of shrinking sea ice."
7,A warning sign that some scientists worry could mean global warming has passed an ominous tipping point.
8,"About half the population of Cambridge, Massachusetts, across the Charles River from Boston and home to Harvard and MIT, could be locked in to a future below sea level by the early 2060s."
9,Precipitation patterns in California will shift with warming.


In [143]:
# Dev data

In [147]:
both_wrong = get_both_wrong(window_1_bad_preds_downsampled_mturk_dev,
                            downsampled_bad_preds_dev)
window1_right = get_m1_right(window_1_preds_downsampled_mturk_dev,
                             window_1_bad_preds_downsampled_mturk_dev,
                            downsampled_bad_preds_dev)
downsampled_right = get_m1_right(downsampled_preds_dev,downsampled_bad_preds_dev,
                            window_1_bad_preds_downsampled_mturk_dev)

In [148]:
print(len(window1_right),len(downsampled_right),len(both_wrong),
      len(window_1_preds_downsampled_mturk_dev),
len(downsampled_preds_dev))

15 47 38 237 237


In [149]:
pd.DataFrame(downsampled_right)

,0
0,"Because climate change perceptions in children seem less susceptible to the influence of worldview or political context, it may be possible for them to inspire adults toward higher levels of climate concern, and in turn, collective action."
1,"The climate benefits of the program -- as well as the health benefits that will come from a reduction in tailpipe pollution, and the jobs and improved infrastructure that fresh investment will bring -- more than outweigh those costs."
2,Progress in avoiding environmental regrets can come through constructive discussion of ways to cut risks and waste and to sustain a long-term energy quest that extends beyond fossil fuels even while they remain abundant and cheap.
3,Cutting down on fossil fuels is a monumental task but doing so ultimately offered economic as well as ecological benefits.
4,Scientists see no end to the way man-made climate change keeps shattering records.
5,Global warming global warming global warming global warming turns out to be as real and present and dangerous as ever it is.
6,A carbon pricing policy is essential if humanity wants to use the resources of creation wisely.
7,The fall-off in ice volume is so fast it is going to bring us to zero very quickly.
8,"While the data are clear, a global political consensus over how to tackle climate change remains elusive."
9,People are linking recent weather phenomena with the concept of global warming.


In [150]:
pd.DataFrame(window1_right)

,0
0,They would look self-serving and out of touch if they tried to deny climate change.
1,"Warmer-than-normal sea surface temperatures are a key player in the development of hurricanes such as Katrina and superstorm Sandy, which hit the U.S. east coast in 2011."
2,Global warming is not expected to end anytime soon.
3,Global warming can increase snowfall by boosting the amount of moisture in the air.
4,Its appropriate to call such storms global warming type events.
5,"The Obama administration new plan to cut carbon pollution from power plants will not just fight climate change, but will also quickly improve public health, preventing up to 100,000 asthma attacks and 2,100 heart attacks in the first year the rules take effect."
6,Well over 90 percent of published climate scientists have concluded that recent global warming is both real and mostly the result of human activity.
7,Climate change significantly increased the amount of rain that flooded Houston.
8,"In the very act of inventing tools to improve the environment, we will come closer to understanding our place on the planet, as well as in a global society of creatures like ourselves."
9,"The behavior of arctic sea ice represents one of the numerous interesting phenomena that the earth presents us with, and for which neither the understanding nor the needed records exist."


In [151]:
pd.DataFrame(both_wrong)

,0
0,A new Ice Age is coming.
1,"It is urgent to move away from fossil fuels and towards renewable energy, including large scale wind and household solar."
2,"The global average sea level rose to a new record high in 2016, and is about 3.25 inches higher than that observed in 1993, when satellite record-keeping for sea level began."
3,"Not acting on climate change is a ""kind of sin,"" as though God had a political program, and that program involved handing power over to Pete Buttigieg."
4,They are too pessimistic about the chances of limiting global warming.
5,The United States government has violated their constitutional rights by allowing fossil fuel production to continue despite knowing its effects on the planet.
6,Its a smoking gun for sea-level rise causing an acceleration of erosion.
7,ExxonMobil for years had understood the climate risks associated with fossil fuels while publicly downplaying the danger.
8,The measure of introducing a carbon tax should be adopted soon.
9,Tweaking a cow's diet to include 2 percent seaweed could reduce its methane emissions by as much as 99 percent.


# GPT(?) predictions 

In [51]:
bad_preds['true_stance'] = bad_preds['true_stance'].apply(to_int)
bad_preds['pred_stance'] = bad_preds['pred_stance'].apply(to_int)
good_preds['true_stance'] = good_preds['true_stance'].apply(to_int)
good_preds['pred_stance'] = good_preds['pred_stance'].apply(to_int)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [46]:
bad_preds = preds[preds.true_stance != preds.pred_stance]
good_preds = preds[preds.true_stance == preds.pred_stance]
opp_preds = bad_preds[bad_preds.true_stance == bad_preds.pred_stance*-1]

In [47]:
bad_preds.shape

(111, 4)

In [48]:
good_preds.shape

(53, 4)

In [54]:
opp_preds.shape

(22, 4)

In [55]:
false_neutrals = bad_preds[bad_preds.pred_stance == 0]
false_agrees = bad_preds[bad_preds.pred_stance == 1]
false_disagrees = bad_preds[bad_preds.pred_stance == -1]
true_neutrals = good_preds[good_preds.pred_stance == 0]
true_agrees = good_preds[good_preds.pred_stance == 1]
true_disagrees = good_preds[good_preds.pred_stance == -1]

27% of predictions were false "agrees". More "neutrals" than "disagrees" were falsely predicted.
There were 19 true agrees.

In [31]:
len(false_agrees)/164

0.27439024390243905

In [41]:
len(true_agrees)

19

In [33]:
false_agrees.true_stance.value_counts()

 0    33
-1    12
Name: true_stance, dtype: int64

13% of predictions were false "disagrees". About equal numbers of "agrees" and "neutrals" were falsely predicted. There were 7 true disagrees. 

In [57]:
len(false_disagrees)/164

0.12804878048780488

In [58]:
len(true_disagrees)

7

In [59]:
false_disagrees.true_stance.value_counts()

0    11
1    10
Name: true_stance, dtype: int64

27% of predictions were false neutrals. More "agrees" than "disagrees" were getting predicted to be "neutral".

In [27]:
len(false_neutrals)/164.

0.27439024390243905

In [28]:
false_neutrals.true_stance.value_counts()

 1    29
-1    16
Name: true_stance, dtype: int64

13% of predictions were the opposite of the correct label (e.g., "agree" was predicted "disagree" or vice versa.

In [61]:
len(opp_preds)/164.

0.13414634146341464

Of this 13%, about half are mistaken "agrees" (and the other 55% mistaken "disagrees").

In [24]:
len(opp_preds[opp_preds.true_stance == 1])/len(opp_preds)

0.45454545454545453

In [65]:
opp_preds

,type,sentence,true_stance,pred_stance
4,mturk,Global warming isn't real Leo.,-1,1
6,mturk,"A combination of factors are at work, including a natural El Nino system, man-made global warming and a dash of random weather.",-1,1
17,mturk,The paper in no way diminishes the extensive body of observations that global warming is happening and that it is largely due to human activity.,1,-1
23,mturk,"Carbon dioxide fertilizes plants, and emissions from fossil fuels have already had a hugely beneficial effect on crops, increasing yields by at least 10-15 percent.",-1,1
34,mturk,Global warming is having an impact now.,1,-1
36,mturk,Man-made climate change isn't solely to blame for tropical storms.,-1,1
37,mturk,Fossil fuel investments are pretty good bets regardless of the political climate.,-1,1
58,mturk,The claim made in the 2007 report by the Intergovernmental Panel on Climate Change (IPCC) is unrealistic and the large Himalayan glaciers could not melt in a few decades.,-1,1
59,mturk,Climate change would be used as an excuse to expand `big government' and lead to a loss of personal freedom.,-1,1
61,mturk,Global warming is real.,1,-1


In [63]:
list(opp_preds.sentence)

["Global warming isn't real Leo.",
 'A combination of factors are at work, including a natural El Nino system, man-made global warming and a dash of random weather.',
 'The paper in no way diminishes the extensive body of observations that global warming is happening and that it is largely due to human activity.',
 'Carbon dioxide fertilizes plants, and emissions from fossil fuels have already had a hugely beneficial effect on crops, increasing yields by at least 10-15 percent.',
 'Global warming is having an impact now.',
 "Man-made climate change isn't solely to blame for tropical storms.",
 'Fossil fuel investments are pretty good bets regardless of the political climate.',
 'The claim made in the 2007 report by the Intergovernmental Panel on Climate Change (IPCC) is unrealistic and the large Himalayan glaciers could not melt in a few decades.',
 "Climate change would be used as an excuse to expand `big government' and lead to a loss of personal freedom.",
 'Global warming is real.'